# Fixture Para Todos

## Cantidad de equipos

Opciones: 4, 6, 8 y 10

In [1]:
n_teams = 8

## Valores parametrizables del algoritmo

### Tiempo máximo de ejecución (en minutos)

Ingresar -1 para que el algoritmo no corte por tiempo

In [2]:
max_execution_time = -1

### Porcentaje de mejora de la varianza

Ingresar -1 para que el algoritmo no corte por porcentaje de mejora

In [3]:
variance_improve_perc = 30

### Cantidad de iteraciones del algoritmo

Ingresar -1 para que el algoritmo no corte por cantidad de iteraciones

In [4]:
n_iterations = 100

In [5]:
heuristic_params = {"max_execution_time": max_execution_time,
                    "variance_improve_perc": variance_improve_perc,
                    "n_iterations": n_iterations}

## Parámetros de las restricciones

### Restricción 1

Todos los equipos deben jugar la misma cantidad de partidos de local y de visitante que el resto o, a lo sumo, <b>param_r1</b> partido/s de diferencia con respecto al resto de los equipos

In [6]:
param_r1 = 1

### Restricción 2

Todos los equipos deben jugar la misma cantidad de partidos de local y de visitante o, a lo sumo, <b>param_r2</b> partido/s de diferencia entre ambas condiciones

In [7]:
param_r2 = 1

### Restricción 3

Ningún equipo puede jugar más de <b>param_r3</b> partidos consecutivos en la misma condición

In [8]:
param_r3 = 2

### Restricción 4

Los equipos considerados clásicos entre sí no pueden jugar de local en la misma fecha

In [9]:
restrictions_params = {"r1": param_r1, "r2": param_r2, "r3": param_r3}

## Importación de librerías y archivos

In [10]:
import csv
import math
import time
from fixture_manager import FixtureManager
from distance_travelled_manager import DistanceTravelledManager

## Carga de archivos CSV

In [11]:
# Carga los equipos a partir del archivo csv
def load_teams(path):
    '''
        Formato de los equipos:
        ["Equipo A", ..., "Equipo Z"]
    '''

    teams = []
    with open(path) as f:
        content = f.readlines()
    teams = [x.strip() for x in content]

    return teams

In [12]:
# Carga las distancias a partir del archivo csv
def load_distances(path):
    '''
        Formato de las distancias:
        {"Equipo A": {"Equipo B": 100, ..., "Equipo Z": 200},
        ...,
        "Equipo X": {"Equipo Y": 150, ..., "Equipo Z": 300}
        }
    '''

    distances = {}
    header = True

    with open(path) as csv_file:
        csv_distances = csv.reader(csv_file, delimiter=",")
        for row in csv_distances:
            if header:
                header = False
                continue

            team1, team2, distance = row[0], row[1], row[2]

            if team1 not in distances:
                distances[team1] = {}
            distances[team1][team2] = distance

            if team2 not in distances:
                distances[team2] = {}
            distances[team2][team1] = distance

    return distances

In [13]:
# Carga los derbies a partir del archivo csv
def load_derbies(path):
    '''
    Formato de los derbies:
        {"Equipo A": "Equipo B",
        ...,
        "Equipo Y": "Equipo Z"
        }
    '''

    derbies = {}
    header = True

    with open(path) as csv_file:
        csv_derbies = csv.reader(csv_file, delimiter=",")
        for row in csv_derbies:
            if header:
                header = False
                continue
            
            team1 = row[0]
            team2 = row[1]
            derbies[team1] = team2
            derbies[team2] = team1

    return derbies

In [14]:
base_path = "csv/" + str(n_teams) + "-teams/"

teams = load_teams(base_path + "teams.csv")
distances = load_distances(base_path + "distances.csv")
derbies = load_derbies(base_path + "derbies.csv")

## Creación del Fixture Manager

In [15]:
fixture_manager = FixtureManager(teams, derbies, restrictions_params, distances, heuristic_params)

## Fixture Inicial

### Generación

In [16]:
start = time.time()
initial_fixture = fixture_manager.calculate_initial_fixture()
end = time.time()

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Talleres: 1901
- Union: 709
- River Plate: 1445
- Newells: 723
- Boca Juniors: 1663
- Rosario Central: 1199
- Belgrano: 1076
- Colon: 998

Varianza de los km recorridos por todos los equipos:
1135.32




In [17]:
print("Tiempo transcurrido: " + str(round(end - start, 2)) + " segundos")

Tiempo transcurrido: 0.3 segundos


In [18]:
fixture_manager.print(initial_fixture)

--------------------------------------------------------------
--------------------- FIXTURE PARA TODOS ---------------------
--------------------------------------------------------------

Fecha 1:
	Rosario Central		Vs.		Talleres
	Colon			Vs.		Union
	Belgrano		Vs.		River Plate
	Boca Juniors		Vs.		Newells

Fecha 2:
	Belgrano		Vs.		Newells
	Rosario Central		Vs.		Union
	River Plate		Vs.		Talleres
	Colon			Vs.		Boca Juniors

Fecha 3:
	River Plate		Vs.		Rosario Central
	Talleres		Vs.		Belgrano
	Newells			Vs.		Colon
	Union			Vs.		Boca Juniors

Fecha 4:
	Colon			Vs.		Talleres
	Newells			Vs.		Union
	Boca Juniors		Vs.		River Plate
	Belgrano		Vs.		Rosario Central

Fecha 5:
	River Plate		Vs.		Colon
	Union			Vs.		Belgrano
	Rosario Central		Vs.		Newells
	Talleres		Vs.		Boca Juniors

Fecha 6:
	Boca Juniors		Vs.		Belgrano
	Colon			Vs.		Rosario Central
	Newells			Vs.		River Plate
	Talleres		Vs.		Union

Fecha 7:
	Union			Vs.		River Plate
	Boca Juniors		Vs.		Rosario Central
	Newells			Vs.		Talleres
	Be

### Validación

In [19]:
print("¿Fixture compatible?")
fixture_manager.is_compatible(initial_fixture)

¿Fixture compatible?


True

### Cálculo de la varianza

In [20]:
initial_variance = fixture_manager.calculate_km_travelled_variance(initial_fixture)
fixture_manager.initial_variance = initial_variance
initial_variance

1135.32

## Fixture Optimizado

In [21]:
start = time.time()
optimized_fixture = fixture_manager.calculate_optimized_fixture()
end = time.time()

ITERACIÓN N° 1

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Colon: 1166
- Talleres: 1513
- River Plate: 2140
- Newells: 864
- Belgrano: 1478
- Union: 843
- Rosario Central: 912
- Boca Juniors: 798

Varianza de los km recorridos por todos los equipos:
1242.29


ITERACIÓN N° 2

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Rosario Central: 757
- Belgrano: 1887
- Colon: 1304
- Boca Juniors: 1354
- Talleres: 1073
- Union: 1289
- Newells: 762
- River Plate: 1288

Varianza de los km recorridos por todos los equipos:
961.65




In [22]:
print("Tiempo transcurrido: " + str(round(end - start, 2)) + " segundos")

Tiempo transcurrido: 0.47 segundos


In [23]:
fixture_manager.print(optimized_fixture)

--------------------------------------------------------------
--------------------- FIXTURE PARA TODOS ---------------------
--------------------------------------------------------------

Fecha 1:
	Union			Vs.		Rosario Central
	Newells			Vs.		Belgrano
	River Plate		Vs.		Colon
	Talleres		Vs.		Boca Juniors

Fecha 2:
	Colon			Vs.		Rosario Central
	River Plate		Vs.		Talleres
	Belgrano		Vs.		Union
	Newells			Vs.		Boca Juniors

Fecha 3:
	Colon			Vs.		Belgrano
	Talleres		Vs.		Newells
	Rosario Central		Vs.		River Plate
	Boca Juniors		Vs.		Union

Fecha 4:
	Boca Juniors		Vs.		Colon
	Newells			Vs.		Rosario Central
	Union			Vs.		Talleres
	Belgrano		Vs.		River Plate

Fecha 5:
	River Plate		Vs.		Union
	Rosario Central		Vs.		Boca Juniors
	Belgrano		Vs.		Talleres
	Colon			Vs.		Newells

Fecha 6:
	Union			Vs.		Newells
	Rosario Central		Vs.		Belgrano
	Boca Juniors		Vs.		River Plate
	Talleres		Vs.		Colon

Fecha 7:
	Boca Juniors		Vs.		Belgrano
	Talleres		Vs.		Rosario Central
	Colon			Vs.		Union
	Newells	

### Validación

In [24]:
print("¿Fixture compatible?")
fixture_manager.is_compatible(optimized_fixture)

¿Fixture compatible?


True

### Cálculo de la varianza

In [25]:
optimized_variance = fixture_manager.calculate_km_travelled_variance(optimized_fixture)
optimized_variance

961.65

### Mejora respecto del fixture inicial

In [26]:
print("Varianza del fixture inicial: " + str(initial_variance) + " km")

Varianza del fixture inicial: 1135.32 km


In [27]:
print("Varianza del fixture final: " + str(optimized_variance) + " km")

Varianza del fixture final: 961.65 km


In [28]:
print("Porcentaje de mejora: " +
      str(round(fixture_manager.calculate_improve_perc(initial_variance, optimized_variance), 2)) + "%")

Porcentaje de mejora: 15.3%


### Descarga del fixture en formato txt

In [29]:
fixture_manager.download(optimized_fixture)